# State Management with StateMachine
This notebook demonstrates how to work with state management using a StateMachine implementation. We'll explore how to create, manage, and control workflow states in a structured way.

## What we'll learn:
- Basic state machine concepts and implementation
- Creating and connecting workflow steps
- Managing state transitions and data flow
- Working with routing and loops in state machines
- Understanding state snapshots and execution flow

In [ ]:
from typing import TypedDict
from lib.state_machine import (
    StateMachine,
    Step,
    EntryPoint,
    Termination,
)

## Basic State Machine Concepts
Let's start with a simple example that demonstrates the core concepts of our state machine:
1. Defining state schema
2. Creating steps
3. Connecting steps
4. Running the workflow

**Creating the Schema and the State Machine**

In [ ]:
class Schema(TypedDict):
    """Schema defining the structure of our state.

    Attributes:
        input: The input value to process
        output: The processed output value
    """
    input: int
    output: int

In [ ]:
# Create our state machine instance
workflow = StateMachine(Schema)

**Defining the logic for Steps**

In [ ]:
def step_input(state: Schema) -> Schema:
    """First step: Increment the input value.

    Args:
        state: Current state containing input value

    Returns:
        Updated state with incremented value in output
    """
    return {"output": state["input"] + 1, "random": 10}



In [ ]:
def step_double(state: Schema) -> Schema:
    """Second step: Double the previous output.

    Args:
        state: Current state containing output from previous step

    Returns:
        Updated state with doubled output value
    """
    return {"output": state["output"] * 2}


**Creating and Connecting Steps**

In [ ]:
entry = EntryPoint()
s1 = Step("input", step_input)
s2 = Step("double", step_double)
termination = Termination()

In [ ]:
workflow.add_steps([entry, s1, s2, termination])

In [ ]:
workflow.connect(entry, s1)
workflow.connect(s1, s2)
workflow.connect(s2, termination)

In [ ]:
workflow.transitions

**Running the Workflow**

In [ ]:
initial_state = {"input": 4}
run_object = workflow.run(initial_state)
run_object

In [ ]:
run_object.snapshots

## Advanced State Management: Routing and Loops
Now we'll explore more complex state management patterns including:
- Conditional routing between steps
- Creating loops in the workflow
- Managing state through multiple iterations

In [ ]:
class CounterSchema(TypedDict):
    """Schema for a counter-based workflow.

    Attributes:
        count: Current counter value
        max_value: Maximum value before termination
    """
    count: int
    max_value: int

In [ ]:
workflow = StateMachine(CounterSchema)

In [ ]:
def increment_counter(state: CounterSchema) -> CounterSchema:
    """Increment the counter value.

    Args:
        state: Current state with counter value

    Returns:
        Updated state with incremented counter
    """
    return {"count": state["count"] + 1}

In [ ]:
# Create steps
entry = EntryPoint()
increment = Step("increment", increment_counter)
termination = Termination()

In [ ]:
workflow.add_steps([entry, increment, termination])

In [ ]:
# Router logic
def check_counter(state: CounterSchema) -> Step:
    """Determine next step based on counter value.

    Args:
        state: Current state with counter and max value

    Returns:
        Next step to execute (increment or terminate)
    """
    if state["count"] >= state["max_value"]:
        return termination
    return increment

In [ ]:
# Connect steps with a loop in increment
workflow.connect(entry, increment)
workflow.connect(increment, [increment, termination], check_counter)

In [ ]:
workflow.transitions

In [ ]:
initial_state = {"count": 0, "max_value": 3}
run_object = workflow.run(initial_state)
run_object

In [ ]:
run_object.snapshots

In [ ]:
run_object.get_final_state()